In [42]:
import pandas as pd
import json

In [43]:
with open("hp_parsed.json", "r") as f:
    result = json.load(f)


dfs_ebs = {}
for key in ["components", "connections"]:
    dfs_ebs[key] = pd.DataFrame.from_dict(result[key], orient="index").sort_index()

In [44]:
with open("hp_tespy.json", "r") as f:
    result = json.load(f)


dfs_tespy = {}
for key in ["components", "connections"]:
    dfs_tespy[key] = pd.DataFrame.from_dict(result[key], orient="index").sort_index()

In [45]:
key = "connections"

dfs_tespy[key]["fluid_group"] = None
dfs_ebs[key]["fluid_group"] = None

groups_compositions = {}

for idx, row in dfs_tespy[key].iterrows():
    rounded_fluids = {
        k: round(v, 4) for k, v in row["mass_composition"].items()
    }
    dfs_tespy[key].loc[idx, "fluid_group"] = f"{rounded_fluids}"

dfs_tespy[key]["dh"] = 0.0
dfs_tespy[key]["ds"] = 0.0
for group in dfs_tespy[key]["fluid_group"].unique():
    mask = dfs_tespy[key]["fluid_group"] == group
    dfs_tespy[key].loc[mask, ["dh", "ds"]] = (dfs_tespy[key].loc[mask, ["h", "s"]].iloc[0] - dfs_tespy[key].loc[mask, ["h", "s"]]).values


groups_compositions = {}

for idx, row in dfs_ebs[key].iterrows():
    if type(row["mass_composition"]) == dict:
        rounded_fluids = {
            k: round(v, 4) for k, v in row["mass_composition"].items()
        }
        dfs_ebs[key].loc[idx, "fluid_group"] = f"{rounded_fluids}"
    else:
        continue

dfs_ebs[key]["dh"] = 0.0
dfs_ebs[key]["ds"] = 0.0
for group in dfs_ebs[key]["fluid_group"].unique():
    if group is not None:
        mask = dfs_ebs[key]["fluid_group"] == group
        dfs_ebs[key].loc[mask, ["dh", "ds"]] = (dfs_ebs[key].loc[mask, ["h", "s"]].iloc[0] - dfs_ebs[key].loc[mask, ["h", "s"]]).values

In [46]:
columns = ["m", "p", "T"]

overlapping_index = list(set(dfs_tespy[key].index.tolist()) & set(dfs_ebs[key].index.tolist()))
diff_to_tespy = (
    dfs_ebs[key].loc[overlapping_index, columns]
    - dfs_tespy[key].loc[overlapping_index, columns]
).abs() / dfs_tespy[key].loc[overlapping_index, columns]
diff_to_ebs = (
    dfs_ebs[key].loc[overlapping_index, columns]
    - dfs_tespy[key].loc[overlapping_index, columns]
).abs() / dfs_tespy[key].loc[overlapping_index, columns]

columns = ["dh", "ds"]
diff_to_ebs[columns] = (
    dfs_ebs[key].loc[overlapping_index, columns]
    - dfs_tespy[key].loc[overlapping_index, columns]
)

In [47]:
diff_to_ebs

,m,p,T,dh,ds
13,0.000204,0.000000e+00,1.867881e-09,-1.742187,-6.174679e-03
11,0.000204,0.000000e+00,4.175678e-14,0.000000,0.000000e+00
30,0.000020,0.000000e+00,4.378639e-10,0.000000,0.000000e+00
12,0.000204,4.174185e-09,1.298952e-05,-1.310877,-7.084586e-04
34,0.000020,0.000000e+00,3.904448e-05,-14.464954,-3.338157e-02
21,0.000000,0.000000e+00,1.639953e-14,0.000000,0.000000e+00
22,0.000000,0.000000e+00,8.754187e-10,0.000090,4.734306e-08
23,0.000000,0.000000e+00,6.663116e-10,-20.136958,-4.673048e-02
31,0.000020,0.000000e+00,2.880911e-10,0.000076,2.171073e-07
32,0.000020,0.000000e+00,5.074101e-09,0.000076,5.984749e-03


In [48]:
assert (diff_to_ebs[["m", "p", "T"]] < 1e-3).all().all()